In [1]:
import random
import pandas as pd
import numpy as np
import h2o
import math

h2o.init()

ERROR:h2opy:Key init.version_check is not a valid config key


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_101"; Java(TM) SE Runtime Environment (build 1.8.0_101-b13); Java HotSpot(TM) 64-Bit Server VM (build 25.101-b13, mixed mode)
  Starting server from /Users/avkashchauhan/Library/Python/2.7/lib/python/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/x7/331tvwcd6p17jj9zdmhnkpyc0000gn/T/tmpw0vW3M
  JVM stdout: /var/folders/x7/331tvwcd6p17jj9zdmhnkpyc0000gn/T/tmpw0vW3M/h2o_avkashchauhan_started_from_python.out
  JVM stderr: /var/folders/x7/331tvwcd6p17jj9zdmhnkpyc0000gn/T/tmpw0vW3M/h2o_avkashchauhan_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster version:,3.11.0.3665
H2O cluster version age:,9 hours
H2O cluster name:,H2O_from_python_avkashchauhan_3ou84q
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


In [2]:
#Let's simulate rows of training data with 10 preditors and a response variable 
#(response will be a logistic function with some noise)

columns = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'R']
data = []
n = 1000000
for i in range(n):
    a = random.randint(0, 1)
    b = random.randint(0, 1)
    c = random.randint(0, 1)
    d = random.randint(0, 1)
    e = random.randint(0, 3)
    f = random.randint(0, 1)
    g = random.randint(0, 1)
    h = random.randint(0, 1)
    i = random.randint(0, 1)
    j = random.randint(0, 1)
    k = 1/(1 + math.exp(-2*a + 3*b + 4*c - 2*d + 6*e + 3.2*f + g + h - 2*i - 4*j + np.random.normal(0, 1)))
    data.append([a, b, c, d, e, f, g, h, i, j, k])
dfp = pd.DataFrame(data, columns=columns)
df = h2o.H2OFrame(dfp)

y = df.col_names[-1]
x = df.col_names[:-1]

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [3]:
#split train/test and make the paramater grid
train, test = df.split_frame([.8], seed=10)
ntrees = [1, 2 , 3 , 4, 5, 6, 7, 8, 9, 10, 20, 40, 80, 160, 320]

In [4]:
#Score using lift
def get_lift(df, predictions, retain = .01, target = y):
    df = df.as_data_frame()
    predictions = predictions.as_data_frame()
    mean = df[target].mean()
    df = pd.concat([df, predictions], axis=1)
    df = df.sort_values('predict', ascending=0)
    df = df[:int(retain*len(df))]
    return df[target].mean()/mean

In [9]:
from h2o.estimators.random_forest import H2ORandomForestEstimator
lift = 0
results = []
bestmodel = None
for depth in ntrees:
    rfmodel = H2ORandomForestEstimator(
    ntrees=depth,
    stopping_rounds=2,
    score_each_iteration=True,
    seed=13)
    
    rfmodel.train(x, y, training_frame=train, validation_frame=test)
    
    predictions = rfmodel.predict(test)
    score = get_lift(test, predictions)
    results.append([depth, score])
    if score > lift:
        lift = score
        bestmodel = rfmodel

drf Model Build progress: |███████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
drf Model Build progress: |█

In [10]:
results = pd.DataFrame(results, columns = ["ntrees", "lift"])
results

,ntrees,lift
0,1,8.540005
1,2,8.540571
2,3,8.540655
3,4,8.540648
4,5,8.540666
5,6,8.540666
6,7,8.540666
7,8,8.540666
8,9,8.540666
9,10,8.540666


In [6]:
varimp = pd.DataFrame(bestmodel.varimp(), columns=["Variable", "Relative Importance", "Scaled Importance", "Percentage"])
varimp.head()

NameError: name 'bestmodel' is not defined

In [8]:
#campaign = 'pj20160249'
#this notebook was completed using campaign = 'pj20160249' but the csv files (train/test/var_desc) sent over are for campaign = 'pj20160400'

campaign = 'pj20160400'
data_path = ''
output_path = 'data/output/'
prior = 0.01
train_file = data_path + campaign + '_train_data.csv'
test_file = data_path + campaign + '_test_data.csv'
var_descr_file = data_path + campaign + '_var_descr.csv'
cutoff_points = [0.01,0.05,0.1,0.2]

In [9]:
import var_reducer as vr
predictors = vr.wrangler(var_descr_file)
train_file

'pj20160400_train_data.csv'

In [10]:
df2 = h2o.import_file(train_file)
df2['trial'] = df2['trial'].asfactor()

x = predictors
y = 'trial'

Parse progress: |█████████████████████████████████████████████████████████████████████████████| 100%


In [11]:
#split train/test and make the paramater grid
train, test = df2.split_frame([.8], seed=10)
ntrees = [10, 20, 30, 40, 50]

In [12]:
from h2o.estimators.random_forest import H2ORandomForestEstimator
lift = 0
results = []
bestmodel = None
for depth in ntrees:
    rfmodel = H2ORandomForestEstimator(
    ntrees=depth,
    stopping_rounds=2,
    score_each_iteration=True,
    seed=13)
    
    rfmodel.train(x, y, training_frame=train, validation_frame=test)
    
    predictions = rfmodel.predict(test)
    score = get_lift(test, predictions, retain= .01, target=y)
    results.append([depth, score])
    if score > lift:
        lift = score
        bestmodel = rfmodel

drf Model Build progress: |███████████████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████████████████████████| 100%
drf prediction progress: |█████████████████████████████████████████████████████████████████

In [13]:
results = pd.DataFrame(results, columns = ["ntrees", "lift"])
results

,ntrees,lift
0,10,1.700338
1,20,1.605874
2,30,1.700338
3,40,1.700338
4,50,1.700338


In [14]:
varimp = pd.DataFrame(bestmodel.varimp(), columns=["Variable", "Relative Importance", "Scaled Importance", "Percentage"])
varimp[:10]

,Variable,Relative Importance,Scaled Importance,Percentage
0,m999999b38p13u,1081.360962,1.000000,0.043615
1,m999999b40p13r,942.651062,0.871727,0.038021
2,m999999b49p13t,892.178711,0.825052,0.035985
3,m999999b38p13d,583.616638,0.539706,0.023539
4,tg_trans,487.302429,0.450638,0.019655
5,m815815b2p13h,472.517029,0.436965,0.019058
6,m999999b49p13d,464.764130,0.429796,0.018746
7,m999999b49p13u,455.960876,0.421655,0.018391
8,m815815b3p13r,348.997223,0.322739,0.014076
9,m999999b38p13h,264.082916,0.244213,0.010651


In [15]:
def chooseKBest(df, k, fnScore, target = -1, x = None):
    if x == None:
        x = df.columns[:]
    if type(target) == 'int':
        y = x[target]
    else:
        y = target
    if y in x:
        x.remove(y)
    scores = fnScore(df, x, y)
    ranked = dict(zip(x, scores))
    return sorted(ranked, key=ranked.get, reverse=True)[:k]

def f_class(df, x, y):
    '''
    This is the method laid out on page 3 of the paper http://www.csie.ntu.edu.tw/~cjlin/papers/features.pdf
    Should match sklearn SelectKBest
    '''
    assert df[y].types[y] is 'enum', '%s is not a categorical value' % (y)
    df_list = [df[df[y] == i][x] for i in df[y].levels()[0]]
    n_features = len(x)
    n_levels_samples = [len(data) for data in df_list]
    n_levels = len(df_list)
    n_samples = sum(n_levels_samples)   
    means = [df_list[i][x].mean() for i in range(n_levels)]
    var = [[df_list[i][j].var() for j in x] for i in range(n_levels)]
    totals = [[df_list[i][j].sum() for j in x] for i in range(n_levels)]
    feature_means = [sum([totals[i][j] for i in range(n_levels)])/n_samples for j in range(n_features)]
    num = [sum([(means[i][j]-feature_means[j])**2 for i in range(n_levels)]) for j in range(n_features)]
    den = [sum(var[i][j]/(n_levels_samples[i]-1) for i in range(n_levels)) for j in range(n_features)]
    scores = [num[i]/den[i] if den[i] > 0.001 else 0 for i in range(n_features)]
    return scores

kbest = chooseKBest(df2, 100, f_class, target='trial', x=predictors)

In [16]:
from h2o.estimators.random_forest import H2ORandomForestEstimator
lift = 0
results = []
bestmodel = None
for depth in ntrees:
    rfmodel = H2ORandomForestEstimator(
    ntrees=depth,
    stopping_rounds=2,
    score_each_iteration=True,
    seed=13)
    
    rfmodel.train(kbest, y, training_frame=train, validation_frame=test)
    
    predictions = rfmodel.predict(test)
    score = get_lift(test, predictions, retain= .01, target=y)
    results.append([depth, score])
    if score > lift:
        lift = score
        bestmodel2 = rfmodel

drf Model Build progress: |███████████████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████████████████████████| 100%
drf prediction progress: |█████████████████████████████████████████████████████████████████

In [17]:
results2 = pd.DataFrame(results, columns = ["ntrees", "lift"])
results2

,ntrees,lift
0,10,1.322485
1,20,1.416948
2,30,1.653106
3,40,1.653106
4,50,1.653106


In [18]:
varimp2 = pd.DataFrame(bestmodel.varimp(), columns=["Variable", "Relative Importance", "Scaled Importance", "Percentage"])
varimp2[:10]

AttributeError: 'NoneType' object has no attribute 'varimp'

In [19]:
important = varimp[:10]['Variable'].as_matrix()
sum([1 for i in important if i in kbest])

10

In [20]:
(1.700338-1.653106)/1.700338

0.027778006490474214